In [1]:
import pandas as pd
import numpy as np

In [2]:
series=pd.read_csv("series.csv", encoding='utf-8')

In [193]:
movies=pd.read_csv("movies.csv",encoding='utf-8')

In [3]:
episodes=pd.read_csv("episodes.csv",encoding='utf-8')

In [118]:
title_episode4 = pd.read_csv('title.episode.tsv', delimiter ='\t', encoding='utf-8')

## Function to compute the IMDB weighted rating for each movie

In [195]:
movies.head()

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,1894,\N,45,Romance,NaN,NaN,NaN,5.4,108.0
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,\N,20,"Documentary,News,Sport",NaN,NaN,NaN,5.2,341.0
2,tt0000335,movie,Soldiers of the Cross,1900,\N,\N,"Biography,Drama",NaN,NaN,NaN,6.1,40.0
3,tt0000502,movie,Bohemios,1905,\N,100,\N,NaN,NaN,NaN,4.4,5.0
4,tt0000574,movie,The Story of the Kelly Gang,1906,\N,70,"Biography,Crime,Drama",NaN,NaN,NaN,6.1,567.0


In [196]:
movies.shape

(668947, 12)

In [197]:
movies["runtimeMinutes"]=movies["runtimeMinutes"].astype("unicode")

In [198]:
movies["runtimeMinutes"]=movies["runtimeMinutes"].replace("\\N",0)

In [199]:
movies["runtimeMinutes"].describe()

count     668947
unique       488
top            0
freq      242228
Name: runtimeMinutes, dtype: int64

In [200]:
runtime=movies["runtimeMinutes"]

In [201]:
movies["runtimeMinutes"].isna().sum()

0

In [202]:
movies=movies[movies.runtimeMinutes!=0]

In [203]:
movies.shape

(426719, 12)

In [204]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426719 entries, 0 to 668946
Data columns (total 12 columns):
tconst            426719 non-null object
titleType         426719 non-null object
originalTitle     426719 non-null object
startYear         426719 non-null object
endYear           426719 non-null object
runtimeMinutes    426719 non-null object
genres            426719 non-null object
parentTconst      0 non-null float64
seasonNumber      0 non-null float64
episodeNumber     0 non-null float64
averageRating     255392 non-null float64
numVotes          255392 non-null float64
dtypes: float64(5), object(7)
memory usage: 42.3+ MB


In [205]:
movies["numVotes"].describe()

count    2.553920e+05
mean     3.193951e+03
std      3.064729e+04
min      5.000000e+00
25%      1.500000e+01
50%      5.400000e+01
75%      2.820000e+02
max      2.219447e+06
Name: numVotes, dtype: float64

In [206]:
movies["averageRating"].isna().sum()

171327

In [207]:
movies=movies[movies["averageRating"].notna()]

In [208]:
movies.shape

(255392, 12)

In [209]:
movies.head()

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,1894,\N,45,Romance,NaN,NaN,NaN,5.4,108.0
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,\N,20,"Documentary,News,Sport",NaN,NaN,NaN,5.2,341.0
3,tt0000502,movie,Bohemios,1905,\N,100,\N,NaN,NaN,NaN,4.4,5.0
4,tt0000574,movie,The Story of the Kelly Gang,1906,\N,70,"Biography,Crime,Drama",NaN,NaN,NaN,6.1,567.0
9,tt0000679,movie,The Fairylogue and Radio-Plays,1908,\N,120,"Adventure,Fantasy",NaN,NaN,NaN,4.8,34.0


In [210]:
movies.drop(["parentTconst","seasonNumber","episodeNumber","titleType","tconst"],axis=1,inplace=True)

In [211]:
movies.shape

(255392, 7)

In [212]:
movies.head()

,originalTitle,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,Miss Jerry,1894,\N,45,Romance,5.4,108.0
1,The Corbett-Fitzsimmons Fight,1897,\N,20,"Documentary,News,Sport",5.2,341.0
3,Bohemios,1905,\N,100,\N,4.4,5.0
4,The Story of the Kelly Gang,1906,\N,70,"Biography,Crime,Drama",6.1,567.0
9,The Fairylogue and Radio-Plays,1908,\N,120,"Adventure,Fantasy",4.8,34.0


In [213]:
C = movies['averageRating'].mean()
C

6.227948408720867

In [217]:
#Calculate the number of votes garnered by the 80th percentile movie
m=movies["numVotes"].quantile(0.81)
m

479.0

In [218]:
new_movies=movies[movies["numVotes"]>=m]

In [219]:
new_movies.shape

(48595, 7)

In [220]:
# Function to compute the IMDB weighted rating for each movie
def weighted_rating(x, m=m, C=C):
    v = x["numVotes"]
    R = x['averageRating']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)

In [221]:
new_movies["score"]=new_movies.apply(weighted_rating,axis=1)

C:\Users\thanu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [222]:
new_movies=new_movies.sort_values('score', ascending=False)

In [223]:
new_movies.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,score
84306,The Shawshank Redemption,1994,\N,142,Drama,9.3,2219447.0,9.299337
159480,Hababam Sinifi,1975,\N,87,"Comedy,Drama",9.3,35354.0,9.258934
51054,The Godfather,1972,\N,175,"Crime,Drama",9.2,1529315.0,9.199069
439358,CM101MMXI Fundamentals,2013,\N,139,"Comedy,Documentary",9.2,42815.0,9.167118
542493,Aynabaji,2016,\N,147,"Crime,Mystery,Thriller",9.2,19095.0,9.127270
414678,Wheels,2014,\N,115,Drama,9.1,17328.0,9.022743
244943,The Dark Knight,2008,\N,152,"Action,Crime,Drama",9.0,2195241.0,8.999395
53388,The Godfather: Part II,1974,\N,202,"Crime,Drama",9.0,1070772.0,8.998761
616701,Peranbu,2018,\N,147,Drama,9.1,11204.0,8.982247
159481,Hababam Sinifi Sinifta Kaldi,1976,\N,91,Comedy,9.0,20599.0,8.937005


In [225]:
new_movies.to_csv("final_movies.csv",index=False)

# Series

In [119]:
title_episode4.shape

(4780646, 4)

In [120]:
title_episode4.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


In [121]:
title_episode4.drop("tconst",axis=1,inplace=True)

In [122]:
title_episode4.rename(columns = {'parentTconst':'tconst'}, inplace = True) 


In [123]:
title_episode4.drop("episodeNumber",axis=1,inplace=True)

In [124]:
title_episode4.head()

,tconst,seasonNumber
0,tt0041038,1
1,tt0989125,1
2,tt0989125,\N
3,tt0040051,3
4,tt0989125,2


In [125]:
title_episode4["seasonNumber"]=title_episode4["seasonNumber"].astype("unicode")


In [126]:
title_episode4.head()

,tconst,seasonNumber
0,tt0041038,1
1,tt0989125,1
2,tt0989125,\N
3,tt0040051,3
4,tt0989125,2


In [127]:
title_episode4["seasonNumber"]=title_episode4["seasonNumber"].replace("\\N",0)


In [128]:
title_episode4=title_episode4[title_episode4.seasonNumber!=0]

In [129]:
title_episode4.head()

,tconst,seasonNumber
0,tt0041038,1
1,tt0989125,1
3,tt0040051,3
4,tt0989125,2
5,tt0989125,2


In [130]:
title_episode4.shape

(3762004, 2)

In [131]:
title_episode4.seasonNumber.min()

'1'

In [132]:
idx=title_episode4.groupby(by='tconst')['seasonNumber'].max()
idx.head()

tconst
tt0039122    1
tt0039123    9
tt0039125    1
tt0040021    2
tt0040028    1
Name: seasonNumber, dtype: object

In [133]:
idx.shape

(120836,)

In [134]:
epi=pd.DataFrame(idx,columns=["seasonNumber"])

In [135]:
epi.head()

,seasonNumber
tconst,
tt0039122,1
tt0039123,9
tt0039125,1
tt0040021,2
tt0040028,1


In [136]:
epi['tconst'] = epi.index
epi.index = range(epi.shape[0])

In [137]:
epi.head()

,seasonNumber,tconst
0,1,tt0039122
1,9,tt0039123
2,1,tt0039125
3,2,tt0040021
4,1,tt0040028


In [138]:
title_episode4.shape

(3762004, 2)

In [139]:
epi.shape


(120836, 2)

In [140]:
series.head(2)

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes
0,tt0035599,series,Voice of Firestone Televues,1943,1947,15,\N,NaN,NaN,NaN,NaN,NaN
1,tt0035803,series,Die Deutsche Wochenschau,1940,1945,12,Documentary,NaN,NaN,NaN,7.8,29.0


In [141]:
series.drop(["seasonNumber","episodeNumber"],axis=1,inplace=True)

In [142]:
series.shape

(212483, 10)

In [143]:
new=pd.merge(series,epi,on="tconst",how="left")

In [144]:
new.shape

(212483, 11)

In [146]:
new.head(5)

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres,parentTconst,averageRating,numVotes,seasonNumber
0,tt0035599,series,Voice of Firestone Televues,1943,1947,15,\N,NaN,NaN,NaN,NaN
1,tt0035803,series,Die Deutsche Wochenschau,1940,1945,12,Documentary,NaN,7.8,29.0,NaN
2,tt0038276,series,You Are an Artist,1946,1950,15,\N,NaN,NaN,NaN,NaN
3,tt0039120,series,Americana,1947,1949,30,"Family,Game-Show",NaN,3.0,8.0,NaN
4,tt0039121,series,Birthday Party,1947,1949,30,Family,NaN,NaN,NaN,NaN


In [147]:
series=new.copy()

In [150]:
series.head(10)

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres,parentTconst,averageRating,numVotes,seasonNumber
0,tt0035599,series,Voice of Firestone Televues,1943,1947,15,\N,NaN,NaN,NaN,NaN
1,tt0035803,series,Die Deutsche Wochenschau,1940,1945,12,Documentary,NaN,7.8,29.0,NaN
2,tt0038276,series,You Are an Artist,1946,1950,15,\N,NaN,NaN,NaN,NaN
3,tt0039120,series,Americana,1947,1949,30,"Family,Game-Show",NaN,3.0,8.0,NaN
4,tt0039121,series,Birthday Party,1947,1949,30,Family,NaN,NaN,NaN,NaN
5,tt0039122,series,The Borden Show,1947,\N,30,"Comedy,Music",NaN,NaN,NaN,1
6,tt0039123,series,Kraft Television Theatre,1947,1958,60,Drama,NaN,8.5,126.0,9
7,tt0039124,series,Party Line,1947,\N,30,\N,NaN,NaN,NaN,NaN
8,tt0039125,series,Public Prosecutor,1947,1951,20,"Crime,Drama,Mystery",NaN,5.9,7.0,1
9,tt0040021,series,Actor's Studio,1948,1950,30,Drama,NaN,7.2,51.0,2


In [151]:
series.drop(["tconst","titleType","parentTconst"],axis=1,inplace=True)

In [152]:
series.shape

(212483, 8)

In [153]:
series["runtimeMinutes"]=series["runtimeMinutes"].astype("unicode")

In [154]:
series["runtimeMinutes"]=series["runtimeMinutes"].replace("\\N",0)

In [155]:
series["runtimeMinutes"].describe()

count     212483
unique       645
top            0
freq      123560
Name: runtimeMinutes, dtype: int64

In [156]:
series["averageRating"].isna().sum()

134345

In [157]:
series=series[series["averageRating"].notna()]

In [158]:
series.shape

(78138, 8)

In [159]:
series_C=series["averageRating"].mean()
series_C

6.7800289231871815

In [160]:
#Calculate the number of votes garnered by the 80th percentile movie
series_m=series["numVotes"].quantile(0.89)
series_m

513.0

In [161]:
series["numVotes"].max()

1659736.0

In [162]:
new_series=series[series["numVotes"]>=series_m]

In [163]:
new_series.shape

(8605, 8)

In [164]:
def weighted_rating_series(x, m=series_m, C=series_C):
    v = x["numVotes"]
    R = x['averageRating']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)

In [165]:
new_series["score"]=new_series.apply(weighted_rating_series,axis=1)

C:\Users\thanu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [166]:
new_series.head()

,originalTitle,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,seasonNumber,score
41,Toast of the Town,1948,1971,60,"Comedy,Music",7.8,1066.0,9,7.468622
98,The Lone Ranger,1949,1957,30,Western,7.8,1924.0,5,7.585291
180,The George Burns and Gracie Allen Show,1950,1958,30,Comedy,8.4,1115.0,8,7.889530
183,The Honeymooners,1955,1956,30,"Comedy,Family",8.6,4431.0,1,8.411156
185,The Jack Benny Program,1950,1965,30,Comedy,8.4,1150.0,9,7.900274


In [167]:
new_series=new_series.sort_values('score', ascending=False)

In [168]:
new_series.head(20)

,originalTitle,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,seasonNumber,score
207525,The Chosen,2017,\N,0,Drama,9.9,4831.0,1,9.600497
182813,Koombiyo,2017,2018,20,"Crime,Drama,Thriller",9.6,15778.0,1,9.511200
46968,Breaking Bad,2008,2013,49,"Crime,Drama,Thriller",9.5,1333424.0,5,9.498954
11356,Band of Brothers,2001,2001,594,"Action,Drama,History",9.5,349450.0,1,9.496013
154958,Planet Earth II,2016,2016,298,Documentary,9.5,86753.0,1,9.484010
136443,The Filthy Frank Show,2011,2017,12,"Comedy,Fantasy,Music",9.5,31554.0,7,9.456487
181861,Chernobyl,2019,2019,330,"Drama,History,Thriller",9.4,443500.0,1,9.396973
43132,Planet Earth,2006,2006,538,Documentary,9.4,158947.0,1,9.391571
60220,Minecraft Epic,2019,\N,0,"Action,Adventure,Comedy",9.8,3230.0,4,9.386095
173723,Blue Planet II,2017,2018,364,Documentary,9.4,28199.0,1,9.353189


In [170]:
new_series.to_csv("final_series.csv",index=False)

## Episodes

In [251]:
episodes.head()

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres,parentTconst,seasonNumber,episodeNumber,averageRating,numVotes
0,tt0041951,episode,The Tenderfeet,1949,\N,30,Western,tt0041038,1,9,7.4,48.0
1,tt0042816,episode,Othello,1950,\N,135,Drama,tt0989125,1,17,NaN,NaN
2,tt0042889,episode,The Tragedy of King Richard II/II,1950,\N,145,Drama,tt0989125,\N,\N,NaN,NaN
3,tt0043426,episode,Coriolanus,1951,\N,60,Drama,tt0040051,3,42,NaN,NaN
4,tt0043631,episode,The Life of King Henry V,1951,\N,133,Drama,tt0989125,2,16,NaN,NaN


In [252]:
episodes.shape

(4780697, 12)

In [253]:
episodes.drop(["parentTconst","titleType","tconst"],axis=1,inplace=True)

In [254]:
episodes["runtimeMinutes"]=episodes["runtimeMinutes"].astype("unicode")

In [255]:
episodes["runtimeMinutes"]=episodes["runtimeMinutes"].replace("\\N",0)

In [256]:
episodes["runtimeMinutes"].describe()

count     4780697
unique        420
top             0
freq      4023437
Name: runtimeMinutes, dtype: int64

In [257]:
episodes["runtimeMinutes"].isna().sum()

0

In [258]:
episodes=episodes[episodes.runtimeMinutes!=0]

In [259]:
episodes.shape

(757260, 9)

In [260]:
episodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757260 entries, 0 to 4780696
Data columns (total 9 columns):
originalTitle     757260 non-null object
startYear         757260 non-null object
endYear           757260 non-null object
runtimeMinutes    757260 non-null object
genres            757253 non-null object
seasonNumber      757253 non-null object
episodeNumber     757253 non-null object
averageRating     291597 non-null float64
numVotes          291597 non-null float64
dtypes: float64(2), object(7)
memory usage: 57.8+ MB


In [261]:
episodes["numVotes"].describe()

count    291597.000000
mean        210.213798
std        1325.335711
min           5.000000
25%          10.000000
50%          28.000000
75%         104.000000
max      219813.000000
Name: numVotes, dtype: float64

In [262]:
episodes["averageRating"].isna().sum()

465663

In [263]:
episodes=episodes[episodes["averageRating"].notna()]

In [264]:
episodes.shape

(291597, 9)

In [265]:
episodes.head()

,originalTitle,startYear,endYear,runtimeMinutes,genres,seasonNumber,episodeNumber,averageRating,numVotes
0,The Tenderfeet,1949,\N,30,Western,1,9,7.4,48.0
7,The Three Musketeers,1950,\N,60,Drama,1,6,4.1,10.0
11,King Lear,1953,\N,75,"Drama,History,Music",2,3,7.0,146.0
13,Othello,1953,\N,84,Drama,\N,\N,8.0,8.0
14,A Christmas Carol,1954,\N,60,"Adventure,Drama,Family",1,4,6.0,121.0


In [266]:
episodes_C =episodes['averageRating'].mean()
episodes_C

7.454938493880582

In [268]:
episodes["numVotes"].max()

219813.0

In [269]:
#Calculate the number of votes garnered by the 80th percentile movie
episodes_m=episodes["numVotes"].quantile(0.9)
episodes_m

389.0

In [270]:
new_episodes=episodes[episodes["numVotes"]>=episodes_m]

In [271]:
new_episodes.shape

(29189, 9)

In [272]:
# Function to compute the IMDB weighted rating for each movie
def weighted_rating_episodes(x, m=episodes_m, C=episodes_C):
    v = x["numVotes"]
    R = x['averageRating']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)

In [273]:
new_episodes["score"]=new_episodes.apply(weighted_rating_episodes,axis=1)

C:\Users\thanu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [290]:
new_episodes=new_episodes.sort_values('score', ascending=False)

In [291]:
new_episodes.head(20)

,originalTitle,startYear,endYear,runtimeMinutes,genres,seasonNumber,episodeNumber,averageRating,numVotes,score
2168804,Ozymandias,2013,\N,48,"Crime,Drama,Thriller",5,14,10.0,113047.0,9.991272
2790176,Battle of the Bastards,2016,\N,60,"Action,Adventure,Drama",6,9,9.9,184660.0,9.894860
2790178,The Winds of Winter,2016,\N,68,"Action,Adventure,Drama",6,10,9.9,128338.0,9.892611
2654043,Hardhome,2015,\N,61,"Action,Adventure,Drama",5,8,9.9,86828.0,9.889095
2090589,The Rains of Castamere,2013,\N,51,"Action,Adventure,Drama",3,9,9.9,86432.0,9.889045
2168808,Felina,2013,\N,55,"Crime,Drama,Thriller",5,16,9.9,81091.0,9.888327
4484419,Vichnaya Pamyat,2019,\N,72,"Drama,History,Thriller",1,5,9.9,49839.0,9.881064
1753981,Face Off,2011,\N,51,"Crime,Drama,Thriller",4,13,9.9,38567.0,9.875585
4776397,Hero,2019,\N,24,"Action,Adventure,Animation",3,17,9.9,24921.0,9.862421
4078040,407 Proxy Authentication Required,2019,\N,56,"Crime,Drama,Thriller",4,7,9.9,16452.0,9.843523


In [292]:
new_episodes.to_csv("final_episodes.csv",index=False)

# EDA 

In [300]:
new_movies=pd.read_csv("final_movies.csv")

In [301]:
new_series=pd.read_csv("final_series.csv")

In [302]:
new_episodes=pd.read_csv("final_episodes.csv")

### Top 10 movies,series, episodes

In [303]:
new_movies.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,score
0,The Shawshank Redemption,1994,\N,142,Drama,9.3,2219447.0,9.299337
1,Hababam Sinifi,1975,\N,87,"Comedy,Drama",9.3,35354.0,9.258934
2,The Godfather,1972,\N,175,"Crime,Drama",9.2,1529315.0,9.199069
3,CM101MMXI Fundamentals,2013,\N,139,"Comedy,Documentary",9.2,42815.0,9.167118
4,Aynabaji,2016,\N,147,"Crime,Mystery,Thriller",9.2,19095.0,9.127270
5,Wheels,2014,\N,115,Drama,9.1,17328.0,9.022743
6,The Dark Knight,2008,\N,152,"Action,Crime,Drama",9.0,2195241.0,8.999395
7,The Godfather: Part II,1974,\N,202,"Crime,Drama",9.0,1070772.0,8.998761
8,Peranbu,2018,\N,147,Drama,9.1,11204.0,8.982247
9,Hababam Sinifi Sinifta Kaldi,1976,\N,91,Comedy,9.0,20599.0,8.937005


In [304]:
new_series.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,seasonNumber,episodeNumber,averageRating,numVotes,score
0,The Chosen,2017,\N,0,Drama,NaN,NaN,9.9,4831.0,9.600497
1,Koombiyo,2017,2018,20,"Crime,Drama,Thriller",NaN,NaN,9.6,15778.0,9.511200
2,Breaking Bad,2008,2013,49,"Crime,Drama,Thriller",NaN,NaN,9.5,1333424.0,9.498954
3,Band of Brothers,2001,2001,594,"Action,Drama,History",NaN,NaN,9.5,349450.0,9.496013
4,Planet Earth II,2016,2016,298,Documentary,NaN,NaN,9.5,86753.0,9.484010
5,The Filthy Frank Show,2011,2017,12,"Comedy,Fantasy,Music",NaN,NaN,9.5,31554.0,9.456487
6,Chernobyl,2019,2019,330,"Drama,History,Thriller",NaN,NaN,9.4,443500.0,9.396973
7,Planet Earth,2006,2006,538,Documentary,NaN,NaN,9.4,158947.0,9.391571
8,Minecraft Epic,2019,\N,0,"Action,Adventure,Comedy",NaN,NaN,9.8,3230.0,9.386095
9,Blue Planet II,2017,2018,364,Documentary,NaN,NaN,9.4,28199.0,9.353189


In [305]:
new_episodes.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,seasonNumber,episodeNumber,averageRating,numVotes,score
0,Ozymandias,2013,\N,48,"Crime,Drama,Thriller",5,14,10.0,113047.0,9.991272
1,Battle of the Bastards,2016,\N,60,"Action,Adventure,Drama",6,9,9.9,184660.0,9.894860
2,The Winds of Winter,2016,\N,68,"Action,Adventure,Drama",6,10,9.9,128338.0,9.892611
3,Hardhome,2015,\N,61,"Action,Adventure,Drama",5,8,9.9,86828.0,9.889095
4,The Rains of Castamere,2013,\N,51,"Action,Adventure,Drama",3,9,9.9,86432.0,9.889045
5,Felina,2013,\N,55,"Crime,Drama,Thriller",5,16,9.9,81091.0,9.888327
6,Vichnaya Pamyat,2019,\N,72,"Drama,History,Thriller",1,5,9.9,49839.0,9.881064
7,Face Off,2011,\N,51,"Crime,Drama,Thriller",4,13,9.9,38567.0,9.875585
8,Hero,2019,\N,24,"Action,Adventure,Animation",3,17,9.9,24921.0,9.862421
9,407 Proxy Authentication Required,2019,\N,56,"Crime,Drama,Thriller",4,7,9.9,16452.0,9.843523


## Maximum votes in which  Genre

In [306]:
movies_genre=new_movies.sort_values('numVotes', ascending=False)
movies_genre.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,score
0,The Shawshank Redemption,1994,\N,142,Drama,9.3,2219447.0,9.299337
6,The Dark Knight,2008,\N,152,"Action,Crime,Drama",9.0,2195241.0,8.999395
24,Inception,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,1946168.0,8.799367
26,Fight Club,1999,\N,139,Drama,8.8,1769695.0,8.799304
13,Pulp Fiction,1994,\N,154,"Crime,Drama",8.9,1741603.0,8.899265
27,Forrest Gump,1994,\N,142,"Drama,Romance",8.8,1712477.0,8.799281
38,The Matrix,1999,\N,136,"Action,Sci-Fi",8.7,1595361.0,8.699258
28,The Lord of the Rings: The Fellowship of the Ring,2001,\N,178,"Action,Adventure,Drama",8.8,1586084.0,8.799223
14,The Lord of the Rings: The Return of the King,2003,\N,201,"Adventure,Drama,Fantasy",8.9,1573582.0,8.899187
2,The Godfather,1972,\N,175,"Crime,Drama",9.2,1529315.0,9.199069


In [307]:
series_genre=new_series.sort_values('numVotes', ascending=False)
series_genre.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,seasonNumber,episodeNumber,averageRating,numVotes,score
11,Game of Thrones,2011,2019,57,"Action,Adventure,Drama",NaN,NaN,9.3,1659736.0,9.299221
2,Breaking Bad,2008,2013,49,"Crime,Drama,Thriller",NaN,NaN,9.5,1333424.0,9.498954
726,The Walking Dead,2010,\N,44,"Drama,Horror,Thriller",NaN,NaN,8.2,822057.0,8.199114
27,Sherlock,2010,\N,88,"Crime,Drama,Mystery",NaN,NaN,9.1,755009.0,9.098425
57,Friends,1994,2004,22,"Comedy,Romance",NaN,NaN,8.9,751489.0,8.898554
83,Stranger Things,2016,\N,51,"Drama,Fantasy,Horror",NaN,NaN,8.8,726480.0,8.798575
927,The Big Bang Theory,2007,2019,22,"Comedy,Romance",NaN,NaN,8.1,684382.0,8.099011
193,Dexter,2006,2013,53,"Crime,Drama,Mystery",NaN,NaN,8.6,619809.0,8.598495
549,How I Met Your Mother,2005,2014,22,"Comedy,Romance",NaN,NaN,8.3,569456.0,8.298632
550,Lost,2004,2010,44,"Adventure,Drama,Fantasy",NaN,NaN,8.3,472474.0,8.298351


In [308]:
episodes_genre=new_episodes.sort_values('numVotes', ascending=False)
episodes_genre.head(10)

,originalTitle,startYear,endYear,runtimeMinutes,genres,seasonNumber,episodeNumber,averageRating,numVotes,score
29176,The Iron Throne,2019,\N,80,"Action,Adventure,Drama",8,6,4.1,219813.0,4.105927
23922,The Long Night,2019,\N,82,"Action,Adventure,Drama",8,3,7.5,199940.0,7.499912
1,Battle of the Bastards,2016,\N,60,"Action,Adventure,Drama",6,9,9.9,184660.0,9.894860
29033,The Bells,2019,\N,78,"Action,Adventure,Drama",8,5,6.0,177605.0,6.003180
29111,The Last of the Starks,2019,\N,78,"Action,Adventure,Drama",8,4,5.5,152482.0,5.504975
2,The Winds of Winter,2016,\N,68,"Action,Adventure,Drama",6,10,9.9,128338.0,9.892611
22329,Winterfell,2019,\N,54,"Action,Adventure,Drama",8,1,7.6,121554.0,7.599537
15420,A Knight of the Seven Kingdoms,2019,\N,58,"Action,Adventure,Drama",8,2,7.9,119673.0,7.898558
0,Ozymandias,2013,\N,48,"Crime,Drama,Thriller",5,14,10.0,113047.0,9.991272
3,Hardhome,2015,\N,61,"Action,Adventure,Drama",5,8,9.9,86828.0,9.889095
